In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
#Creamos una variable para crear la conexión.
conn = psycopg2.connect(
    dbname = "hoteles_bbdd",
    user = "my_user",
    password = "admin",
    host = "localhost",
    port = "5432"
)

In [3]:
cur = conn.cursor()

In [4]:
cur.execute("SELECT version();")
cur.fetchone()

('PostgreSQL 14.12 (Homebrew) on aarch64-apple-darwin23.4.0, compiled by Apple clang version 15.0.0 (clang-1500.3.9.4), 64-bit',)

### Ciudad


In [14]:
df = pd.read_csv("../data/limpieza_reservas_hoteles.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  int64  
 1   id_cliente       15098 non-null  int64  
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   inicio_estancia  15098 non-null  object 
 7   final_estancia   15098 non-null  object 
 8   id_hotel         15098 non-null  int64  
 9   ciudad           15098 non-null  object 
 10  id               15098 non-null  int64  
 11  nombre_hotel     15098 non-null  object 
 12  valoracion       15098 non-null  float64
 13  precio_noche     15098 non-null  float64
 14  fecha_reserva    15098 non-null  object 
 15  id_ciudad        15098 non-null  int64  
dtypes: bool(1), float64(2), int64(5), object(8)
memory usage: 

In [32]:
tabla_ciudad = df["ciudad"]
tabla_ciudad.unique()

array(['Madrid'], dtype=object)

In [27]:
tabla_ciudad = pd.DataFrame(df["ciudad"].unique(),
                          columns=["nombre_ciudad"])

tabla_ciudad

,nombre_ciudad
0,Madrid


In [28]:
insert_query = """
        INSERT INTO ciudad (nombre_ciudad)
        VALUES (%s)
"""

In [29]:
data_to_insert = [(row['nombre_ciudad'],) for _, row in tabla_ciudad.iterrows()]
data_to_insert

[('Madrid',)]

In [30]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [285]:
conn.rollback()

### EVENTOS

CREATE TABLE eventos (
    id_evento SERIAL PRIMARY KEY,
    nombre_evento TEXT,
    url_evento TEXT,
    codigo_postal INT,
    direccion TEXT,
    horario TEXT,
    fecha_inicio DATE,
    fecha_fin DATE,
    organizacion TEXT,
    id_ciudad INT REFERENCES 
    ciudad(id_ciudad) ON DELETE CASCADE
);

In [33]:
tabla_eventos = pd.read_csv("../data/eventos.csv")
tabla_eventos.head()

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,fecha_inicio,fecha_fin,ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005.0,CALLE SAN JUSTO 5,NaN,Biblioteca Pública Municipal Iván de Vargas (C...,2025-02-24,2025-03-02,Madrid
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009.0,PASEO COLOMBIA 1,NaN,Centro Cultural Casa de Vacas (Retiro),2025-02-27,2025-03-23,Madrid
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045.0,PLAZA LEGAZPI 8,NaN,Matadero Madrid,2024-10-01,2025-06-01,Madrid
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,NaN,NaN,NaN,NaN,2024-10-14,2025-06-30,NaN
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029.0,AVENIDA MONFORTE DE LEMOS 38,17:00,Biblioteca Pública Municipal José Saramago (Fu...,2025-01-14,2025-06-17,Madrid


In [34]:
#Añadimos las columnas que faltan (diudad_id, id_evento y organicacion)
tabla_eventos["id_ciudad"] = 1
tabla_eventos["organizacion"]
tabla_eventos["fecha_inicio"] = pd.to_datetime(tabla_eventos["fecha_inicio"]).dt.date
tabla_eventos["fecha_fin"] = pd.to_datetime(tabla_eventos["fecha_fin"]).dt.date
tabla_eventos["codigo_postal"] = pd.to_numeric(tabla_eventos["codigo_postal"], errors="coerce").astype("Int64")


In [35]:
tabla_eventos

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,fecha_inicio,fecha_fin,ciudad,id_ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,NaN,Biblioteca Pública Municipal Iván de Vargas (C...,2025-02-24,2025-03-02,Madrid,1
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,PASEO COLOMBIA 1,NaN,Centro Cultural Casa de Vacas (Retiro),2025-02-27,2025-03-23,Madrid,1
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,NaN,Matadero Madrid,2024-10-01,2025-06-01,Madrid,1
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,<NA>,NaN,NaN,NaN,2024-10-14,2025-06-30,NaN,1
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,17:00,Biblioteca Pública Municipal José Saramago (Fu...,2025-01-14,2025-06-17,Madrid,1
...,...,...,...,...,...,...,...,...,...,...
249,"Y tú, ¿cómo viajas?",http://www.madrid.es/sites/v/index.jsp?vgnextc...,28037,CALLE HERMANOS GARCIA NOBLEJAS 14,10:00,Biblioteca Pública Municipal Ciudad Lineal (Ci...,2024-12-04,2025-05-28,Madrid,1
250,Yoga para el buen trato II,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PASEO CHOPERA 10,18:00,Espacio de Igualdad Juana Doña. Arganzuela,2025-02-18,2025-03-11,Madrid,1
251,'Lina en el mundo de Paul Klee' por Alejandra ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,08:30,Biblioteca Pública Municipal José Saramago (Fu...,2025-02-04,2025-03-25,Madrid,1
252,'Refugiados' por Myriam Laguía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28019,CALLE ISAAC ALBENIZ 1,19:00,Biblioteca Pública Municipal Ana María Matute ...,2025-02-14,2025-03-13,Madrid,1


In [36]:
tabla_eventos = tabla_eventos[['nombre_evento', 'url_evento','codigo_postal', 'direccion', 'horario', 'fecha_inicio', 'fecha_fin', 'organizacion', 'id_ciudad']]
tabla_eventos

,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion,id_ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,NaN,2025-02-24,2025-03-02,Biblioteca Pública Municipal Iván de Vargas (C...,1
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,PASEO COLOMBIA 1,NaN,2025-02-27,2025-03-23,Centro Cultural Casa de Vacas (Retiro),1
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,NaN,2024-10-01,2025-06-01,Matadero Madrid,1
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,<NA>,NaN,NaN,2024-10-14,2025-06-30,NaN,1
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,17:00,2025-01-14,2025-06-17,Biblioteca Pública Municipal José Saramago (Fu...,1
...,...,...,...,...,...,...,...,...,...
249,"Y tú, ¿cómo viajas?",http://www.madrid.es/sites/v/index.jsp?vgnextc...,28037,CALLE HERMANOS GARCIA NOBLEJAS 14,10:00,2024-12-04,2025-05-28,Biblioteca Pública Municipal Ciudad Lineal (Ci...,1
250,Yoga para el buen trato II,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PASEO CHOPERA 10,18:00,2025-02-18,2025-03-11,Espacio de Igualdad Juana Doña. Arganzuela,1
251,'Lina en el mundo de Paul Klee' por Alejandra ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,08:30,2025-02-04,2025-03-25,Biblioteca Pública Municipal José Saramago (Fu...,1
252,'Refugiados' por Myriam Laguía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28019,CALLE ISAAC ALBENIZ 1,19:00,2025-02-14,2025-03-13,Biblioteca Pública Municipal Ana María Matute ...,1


In [37]:
tabla_eventos["codigo_postal"] = tabla_eventos["codigo_postal"].astype("Int64").astype(object).where(pd.notna(tabla_eventos["codigo_postal"]), None)


/var/folders/_s/kc_rj_092690fgthyqft8hk40000gn/T/ipykernel_1127/3653157532.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_eventos["codigo_postal"] = tabla_eventos["codigo_postal"].astype("Int64").astype(object).where(pd.notna(tabla_eventos["codigo_postal"]), None)


In [38]:
tabla_eventos = tabla_eventos.where(pd.notna(tabla_eventos), None)

In [39]:
# Ver los tipos de datos en cada columna
print(tabla_eventos.dtypes)

# Verificar si hay valores <NA> en alguna columna
print(tabla_eventos.isna().sum())

# Ver algunas filas con valores nulos
print(tabla_eventos[tabla_eventos.isna().any(axis=1)])

nombre_evento    object
url_evento       object
codigo_postal    object
direccion        object
horario          object
fecha_inicio     object
fecha_fin        object
organizacion     object
id_ciudad         int64
dtype: object
nombre_evento     0
url_evento        0
codigo_postal    30
direccion        30
horario          89
fecha_inicio      0
fecha_fin         0
organizacion     28
id_ciudad         0
dtype: int64
                                         nombre_evento  \
0              25º aniversario de la revista La Fragua   
1         60 Premio Reina Sofía de Pintura y Escultura   
2                                          A toda vela   
3            Acompañamiento digital a personas mayores   
6    Actividades mensuales de los centros de mayore...   
..                                                 ...   
243                     Visitas a Matadero para grupos   
244  Visitas dialogadas. Fuentesal Arenillas e Itzi...   
245                     Visitas guiadas a CentroCentro 

In [40]:
insert_query = """
        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [41]:
data_to_insert = [
    (
        row["nombre_evento"], 
        row["url_evento"], 
        row["codigo_postal"], 
        row["direccion"], 
        row["horario"], 
        row["fecha_inicio"], 
        row["fecha_fin"], 
        row["organizacion"],
        row["id_ciudad"]
    )
    for _, row in tabla_eventos.iterrows()
]

In [42]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:
conn.rollback()

### HOTELES

CREATE TABLE hoteles (
    id_hotel SERIAL PRIMARY KEY, -- Puede ser VARCHAR(5)
    nombre_hotel TEXT,
    competencia BOOL,
    valoracion FLOAT CHECK (valoracion BETWEEN 1 AND 5),
    id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE
);

In [43]:
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,inicio_estancia,final_estancia,id_hotel,ciudad,id,nombre_hotel,valoracion,precio_noche,fecha_reserva,id_ciudad
0,1,3043,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-03-01,2025-03-02,27,Madrid,1,ibis budget Madrid Centro Lavapies,4.3,106.0,2025-02-19,1
1,2,3043,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-03-01,2025-03-02,23,Madrid,2,ibis Styles Madrid Centro Maravillas,4.7,114.0,2025-02-19,1
2,3,3946,Judith,Rivas,judith.rivas@example.com,True,2025-03-01,2025-03-02,25,Madrid,8,ibis budget Madrid Calle 30,4.4,79.0,2025-02-19,1
3,4,13108,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-03-01,2025-03-02,21,Madrid,10,ibis Madrid Centro las Ventas,4.5,90.0,2025-02-19,1
4,5,2393,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-03-01,2025-03-02,28,Madrid,12,ibis budget Madrid Centro las Ventas,4.3,75.0,2025-02-19,1


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  int64  
 1   id_cliente       15098 non-null  int64  
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   inicio_estancia  15098 non-null  object 
 7   final_estancia   15098 non-null  object 
 8   id_hotel         15098 non-null  int64  
 9   ciudad           15098 non-null  object 
 10  id               15098 non-null  int64  
 11  nombre_hotel     15098 non-null  object 
 12  valoracion       15098 non-null  float64
 13  precio_noche     15098 non-null  float64
 14  fecha_reserva    15098 non-null  object 
 15  id_ciudad        15098 non-null  int64  
dtypes: bool(1), float64(2), int64(5), object(8)
memory usage: 

In [ ]:
CREATE TABLE hoteles (
    id_hotel SERIAL PRIMARY KEY, -- Puede ser VARCHAR(5)
    nombre_hotel TEXT,
    competencia BOOL,
    valoracion FLOAT CHECK (valoracion BETWEEN 1 AND 5),
    id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE
);

In [47]:
tabla_hoteles = df[["id_hotel","nombre_hotel","competencia","valoracion","id_ciudad"]]
tabla_hoteles

,id_hotel,nombre_hotel,competencia,valoracion,id_ciudad
0,27,ibis budget Madrid Centro Lavapies,True,4.3,1
1,23,ibis Styles Madrid Centro Maravillas,True,4.7,1
2,25,ibis budget Madrid Calle 30,True,4.4,1
3,21,ibis Madrid Centro las Ventas,True,4.5,1
4,28,ibis budget Madrid Centro las Ventas,True,4.3,1
...,...,...,...,...,...
15093,7,Hotel Las Estrellas,False,3.0,1
15094,1,Gran Hotel Madrid,False,4.0,1
15095,6,Hotel Jardines del Rey,False,2.0,1
15096,13,Hotel Palacio Imperial,False,1.0,1


In [48]:
#Eliminamos los hoteles que tienen un nombre_hotel duplicado:
tabla_hoteles = tabla_hoteles.drop_duplicates(subset=["nombre_hotel"], keep="first")
tabla_hoteles

,id_hotel,nombre_hotel,competencia,valoracion,id_ciudad
0,27,ibis budget Madrid Centro Lavapies,True,4.3,1
1,23,ibis Styles Madrid Centro Maravillas,True,4.7,1
2,25,ibis budget Madrid Calle 30,True,4.4,1
3,21,ibis Madrid Centro las Ventas,True,4.5,1
4,28,ibis budget Madrid Centro las Ventas,True,4.3,1
5,24,ibis Styles Madrid City Las Ventas,True,4.6,1
7,26,ibis budget Madrid Calle Alcalá,True,4.3,1
8,20,ibis Madrid Calle Alcalá,True,4.5,1
12,29,ibis budget Madrid Vallecas,True,4.3,1
22,22,ibis Madrid Getafe,True,4.4,1


In [49]:
tabla_hoteles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 0 to 5214
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      29 non-null     int64  
 1   nombre_hotel  29 non-null     object 
 2   competencia   29 non-null     bool   
 3   valoracion    29 non-null     float64
 4   id_ciudad     29 non-null     int64  
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 1.2+ KB


In [50]:
data_to_insert = [
    (
        row["id_hotel"], 
        row["nombre_hotel"], 
        row["competencia"], 
        row["valoracion"],
        row["id_ciudad"]
    )
    for _, row in tabla_hoteles.iterrows()
]

In [51]:
insert_query = """
        INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
        VALUES (%s, %s, %s, %s, %s)
"""

In [52]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:

conn.rollback()

### CLIENTES

In [ ]:
CREATE TABLE clientes (
    id_cliente VARCHAR(50) PRIMARY KEY,
    nombre TEXT,
    apellido TEXT,
    mail TEXT UNIQUE CHECK (mail LIKE '%@%')
);

In [15]:
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,inicio_estancia,final_estancia,id_hotel,ciudad,id,nombre_hotel,valoracion,precio_noche,fecha_reserva,id_ciudad
0,1,5311,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-03-01,2025-03-02,27,Madrid,1,ibis budget Madrid Centro Lavapies,4.3,106.0,2025-02-19,1
1,2,8310,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-03-01,2025-03-02,23,Madrid,2,ibis Styles Madrid Centro Maravillas,4.7,114.0,2025-02-19,1
2,3,8047,Judith,Rivas,judith.rivas@example.com,True,2025-03-01,2025-03-02,25,Madrid,8,ibis budget Madrid Calle 30,4.4,79.0,2025-02-19,1
3,4,204,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-03-01,2025-03-02,21,Madrid,10,ibis Madrid Centro las Ventas,4.5,90.0,2025-02-19,1
4,5,9762,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-03-01,2025-03-02,28,Madrid,12,ibis budget Madrid Centro las Ventas,4.3,75.0,2025-02-19,1


In [16]:
tabla_clientes = df[["id_cliente", "nombre","apellido","mail"]]
tabla_clientes

,id_cliente,nombre,apellido,mail
0,5311,Feliciana,Cantón,feliciana.cantón@example.com
1,8310,Leonardo,Hierro,leonardo.hierro@example.com
2,8047,Judith,Rivas,judith.rivas@example.com
3,204,Adolfo,Cuervo,adolfo.cuervo@example.com
4,9762,María Carmen,Navarro,maría carmen.navarro@example.com
...,...,...,...,...
15093,14320,Victoriano,Miralles,victoriano.miralles@example.com
15094,1082,Anacleto,Toledo,anacleto.toledo@example.com
15095,10444,Máxima,Mayoral,máxima.mayoral@example.com
15096,11950,Rafa,Alcaraz,rafa.alcaraz@example.com


In [17]:
#Vemos cuántos valores únicos hay de las columnas id_cliente y mail:
print(tabla_clientes["id_cliente"].nunique())
print(tabla_clientes["mail"].nunique())

14905
14905


In [22]:
#Eliminamos duplicados en base a id_cliente y mail:
tabla_clientes = tabla_clientes.drop_duplicates(subset=["id_cliente", "mail"], keep="first")
tabla_clientes

,id_cliente,nombre,apellido,mail
0,5311,Feliciana,Cantón,feliciana.cantón@example.com
1,8310,Leonardo,Hierro,leonardo.hierro@example.com
2,8047,Judith,Rivas,judith.rivas@example.com
3,204,Adolfo,Cuervo,adolfo.cuervo@example.com
4,9762,María Carmen,Navarro,maría carmen.navarro@example.com
...,...,...,...,...
14995,232,Adoración,Cuadrado,adoración.cuadrado@example.com
14996,4100,Edgardo,Sanchez,edgardo.sanchez@example.com
14997,8630,Lorena,Román,lorena.román@example.com
14998,10526,Nadia,Verdugo,nadia.verdugo@example.com


In [23]:
data_to_insert = [
    (
        row["id_cliente"], 
        row["nombre"], 
        row["apellido"], 
        row["mail"]
    )
    for _, row in tabla_clientes.iterrows()
]

In [24]:
insert_query = """  
        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
        VALUES (%s, %s, %s, %s)
"""

In [26]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [25]:
conn.rollback()

### RESERVAS

In [ ]:
CREATE TABLE reservas (
    id_reserva VARCHAR(50) PRIMARY KEY,
    fecha_reserva DATE,
    inicio_estancia DATE,
    final_estancia DATE,
    precio_noche FLOAT CHECK (precio_noche >= 0),
    id_cliente VARCHAR(50) REFERENCES clientes(id_cliente) ON DELETE CASCADE,
    id_hotel INT REFERENCES hoteles(id_hotel) ON DELETE CASCADE
    
);

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  int64  
 1   id_cliente       15098 non-null  int64  
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   inicio_estancia  15098 non-null  object 
 7   final_estancia   15098 non-null  object 
 8   id_hotel         15098 non-null  int64  
 9   ciudad           15098 non-null  object 
 10  id               15098 non-null  int64  
 11  nombre_hotel     15098 non-null  object 
 12  valoracion       15098 non-null  float64
 13  precio_noche     15098 non-null  float64
 14  fecha_reserva    15098 non-null  object 
 15  id_ciudad        15098 non-null  int64  
dtypes: bool(1), float64(2), int64(5), object(8)
memory usage: 

In [29]:
tabla_reservas = df[["id_reserva","fecha_reserva","inicio_estancia","final_estancia","precio_noche", "id_cliente", "id_hotel"]]
tabla_reservas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  int64  
 1   fecha_reserva    15098 non-null  object 
 2   inicio_estancia  15098 non-null  object 
 3   final_estancia   15098 non-null  object 
 4   precio_noche     15098 non-null  float64
 5   id_cliente       15098 non-null  int64  
 6   id_hotel         15098 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 825.8+ KB


In [34]:
#pasamos a datetime las columnas de fecha_reserva, inicio_estancia y final_estancia:
tabla_reservas["fecha_reserva"] = pd.to_datetime(tabla_reservas["fecha_reserva"])
tabla_reservas["inicio_estancia"] = pd.to_datetime(tabla_reservas["inicio_estancia"])
tabla_reservas["final_estancia"] = pd.to_datetime(tabla_reservas["final_estancia"])

/var/folders/_s/kc_rj_092690fgthyqft8hk40000gn/T/ipykernel_2781/4091136663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_reservas["fecha_reserva"] = pd.to_datetime(tabla_reservas["fecha_reserva"])
/var/folders/_s/kc_rj_092690fgthyqft8hk40000gn/T/ipykernel_2781/4091136663.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_reservas["inicio_estancia"] = pd.to_datetime(tabla_reservas["inicio_estancia"])
/var/folders/_s/kc_rj_092690fgthyqft8hk40000gn/T/ipykernel_2781/4091136663.py:4: SettingWi

In [35]:
tabla_reservas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  int64         
 1   fecha_reserva    15098 non-null  datetime64[ns]
 2   inicio_estancia  15098 non-null  datetime64[ns]
 3   final_estancia   15098 non-null  datetime64[ns]
 4   precio_noche     15098 non-null  float64       
 5   id_cliente       15098 non-null  int64         
 6   id_hotel         15098 non-null  int64         
dtypes: datetime64[ns](3), float64(1), int64(3)
memory usage: 825.8 KB


In [36]:
insert_query = """
        INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche,id_cliente, id_hotel)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [37]:
data_to_insert = [
    ( 
        row["id_reserva"], 
        row["fecha_reserva"], 
        row["inicio_estancia"], 
        row["final_estancia"], 
        row["precio_noche"], 
        row["id_cliente"], 
        row["id_hotel"]
    )
    for _, row in tabla_reservas.iterrows()
]

In [38]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [365]:
conn.rollback()